##1. Сбор корпуса

Импортируем нужные библиотеки

In [ ]:
!pip install fake_useragent

In [ ]:
import requests

import time
from datetime import datetime

from fake_useragent import UserAgent

import re

import random

from bs4 import BeautifulSoup

from tqdm.auto import tqdm

In [ ]:
session = requests.session()
ua = UserAgent()

Собираем ссылки на стаьи о сериях

In [ ]:
req = session.get('https://losyash-library.fandom.com/ru/wiki/%D0%A1%D0%BC%D0%B5%D1%88%D0%B0%D1%80%D0%B8%D0%BA%D0%B8_(%D0%BC%D1%83%D0%BB%D1%8C%D1%82%D1%81%D0%B5%D1%80%D0%B8%D0%B0%D0%BB)')
page = req.text
soup = BeautifulSoup(page, 'html.parser')
links = soup.find_all('a')
episodes = []
for link in links:
  if 'href' in link.attrs.keys():
    episodes.append(link.attrs['href'])

Оставляем только нужные

In [ ]:
eps_list = []
for episode in episodes:
  if episode[:9] == '/ru/wiki/' and not re.search(r'_\d+\._', episode):
    eps_list.append('https://losyash-library.fandom.com' + episode)

Парсинг страницы с цитатами

In [ ]:
def get_quotes(ep_url):
    req = session.get(ep_url)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    allquotes = soup.find_all('table', {'style': 'margin:auto; border-collapse:collapse; background-color:transparent; border-style:none;'})
    if len(allquotes):
      try:
        eptitle = soup.title.text.split(' | ')[0]
      except Exception as e:
        eptitle = soup.title.text
      epquotes = [eptitle]
      for quote in allquotes:
        epquotes.append(quote.text)
      return epquotes
    return False

In [ ]:
episodes = []
links_counter = tqdm(total=len(eps_list), desc='Ссылок обработано')
for i in range(len(eps_list)):
  theepquotes = get_quotes(eps_list[i])
  if theepquotes:
    episodes.append(theepquotes)
  links_counter.update(1)
  time.sleep(random.uniform(3.9, 5.1))

Ссылок обработано:   0%|          | 0/233 [00:00<?, ?it/s]

Заменяем подстроки, которые ломают деление на фразы

In [ ]:
replacements = [['Бараш (Нюше):', 'Бараш:'],
['Бараш (Ёжику):', 'Бараш:'],
 ['Крош (видит бочки):', 'Крош:'],
 ['Ёжик (Крошу):', 'Ёжик:'],
 ['Крош; Смотри, ты шёл со скоростью… беспечного ёжика.Ёжик:', 'Крош: Смотри, ты шёл со скоростью… беспечного ёжика.Ёжик:'],
 ['Кар-Карыч (Нюше в костюме Бараша):', 'Кар-Карыч:'],
 ['Крош А что это такое?Копатыч:', 'Крош: А что это такое?Копатыч:'],
 ['Нюша Выбить пыль, не нарушая поверхности ворса.Крош:', 'Нюша: Выбить пыль, не нарушая поверхности ворса.Крош:'],
 ['Крош Копатычу:', 'Крош:'],
 [' :', ':'], [':К', ': К'], [':У', ': У'], [':Т', ': Т'],
 ['Но в целом мы далеко не идальныеКрош', 'но в целом мы далеко не идальные.Крош'],
 ['Бесполезно, он захлопнулсяЛосяш', 'бесполезно, он захлопнулся.Лосяш'],
 ['Чем больше в лабиринте тупичков, тем он сложней и интереснейКопатыч','чем больше в лабиринте тупичков, тем он сложней и интересней.Копатыч'],
 ['Ну так, Бараш, конечно:', 'ну так, Бараш, конечно — '],
 ['Потребляйт дрова, превращает их в энергия, и пожалуйста:',
  'потребляйт дрова, превращает их в энергия, и пожалуйста — '],
 ['Так и запишем:', 'так и запишем — '],
 ['Она сидит там одна, переживает, думает:',
  'она сидит там одна, переживает, думает — '],
 ['Ну, первое и основное:', 'ну, первое и основное — '],
 ['Едешь в вагоне:', 'едешь в вагоне — '],
 ['И Крош подумал:', 'и Крош подумал — '],
 ['К тому же можно спокойно смотреть по сторонам:',
  'к тому же можно спокойно смотреть по сторонам — '],
 ['Например, говорить слова наоборот:',
  'например, говорить слова наоборот — '],
 ['Ну, например:', 'ну, например — '],
 ['Посмотрел на пирамидку - и сразу же мысль:',
  'посмотрел на пирамидку - и сразу же мысль — '],
 ['Так вот, чтобы исправить:', 'так вот, чтобы исправить — '],
 ['Я же говорил:', 'я же говорил — '],
 ['Но ты не беспокойся:', 'но ты не беспокойся — '],
 ['Вижу:', 'вижу — '],
 ['Повторяю:', 'повторяю — '],
 ['Нет, ты только подумай:', 'нет, ты только подумай — '],
 ['Душа у меня разрывается, душечка, не знаю прямо, что завтра делать:',
  'душа у меня разрывается, душечка, не знаю прямо, что завтра делать — '],
 ['Сбегайте кто-нибудь за приказчиком, от вас, я вижу, толку мало:',
  'сбегайте кто-нибудь за приказчиком, от вас, я вижу, толку мало — '],
 ['Но ничего:', 'но ничего — '],
 ['Это ты:', 'это ты — '],
 ['Да не так уж много всего вокруг:', 'да не так уж много всего вокруг — '],
 ['Посмотри на себя:', 'посмотри на себя — '],
 ['Давай так:', 'давай так — '],
 ['Я ему это объясняю, а он не понимает, говорит:',
  'я ему это объясняю, а он не понимает, говорит — '],
 ['Послушай меня:', 'послушай меня — '],
 ['Ты, кстати, можешь радоваться:', 'ты, кстати, можешь радоваться — '],
 ['Со мной, например, всегда так:', 'со мной, например, всегда так — '],
 ['Ёжик, если бы тебе предложили на выбор:',
  'ёжик, если бы тебе предложили на выбор — '],
 ['Господин режиссёр, я прошу вас:', 'господин режиссёр, я прошу вас — '],
 ['Я знал одного доктора наук, ему тоже всё время говорили:',
  'я знал одного доктора наук, ему тоже всё время говорили — '],
 ['Мы его  на дрова несём:', 'мы его  на дрова несём — '],
 ['Крош, Ёжик, у меня к вам просьба:', 'крош, Ёжик, у меня к вам просьба — '],
 ['Оказывается в мире всё время так происходит:',
  'оказывается в мире всё время так происходит — '],
 ['Ты застудишь ноги, на ногах все нервные окончания, то есть, с ногами ты застудишь обязательно что-нибудь ещё:',
  'ты застудишь ноги, на ногах все нервные окончания, то есть, с ногами ты застудишь обязательно что-нибудь ещё — '],
 ['Ещё бы поспорили, что лучше:', 'ещё бы поспорили, что лучше — ']]

In [ ]:
for i in range(len(episodes)):
  for j in range(1, len(episodes[i])):
    for error in replacements:
      if error[0] in episodes[i][j]:
        episodes[i][j] = episodes[i][j].replace(error[0], error[1])

Удалем мусор

In [ ]:
episodes.remove(['Тайна древних сокровищ', '\n\n\n\nИзобретатель А. С. Попов\n\n\n'])
episodes.remove(['Тайна древних сокровищ', '\n\n\n\nИзобретатель А. С. Попов\n\n\n'])

Записываем собранное

In [ ]:
with open('corpus.txt', 'w') as file:
  for i in range(len(episodes)):
    for j in range(len(episodes[i])):
      file.write(episodes[i][j].strip() + '\n')
    file.write('\n')

Создаем список токенов; всем персонажам и сериям причисляем айди, чтобы хранить его, а не целые строки

In [ ]:
text_ids = {}
cur_id = 0
char_ids = {}
char_ids_map = {}
cur_chid = 0
dialogues = []
for i in range(len(episodes)):
  for j in range(1, len(episodes[i])):
    episodes[i][j] = episodes[i][j].strip()
    if episodes[i][0] + '_' + str(j) in text_ids.values():
      continue
    text_ids[cur_id] = episodes[i][0] + '_' + str(j)
    if '— ' in episodes[i][j]:
      quoteparams = episodes[i][j].split('— ')
      if quoteparams[1] not in char_ids.values():
        char_ids[cur_chid] = quoteparams[1]
        char_ids_map[quoteparams[1]] = cur_chid
        cur_chid = cur_chid + 1
      dialogues.append([[quoteparams[0], cur_id, char_ids_map[quoteparams[1]]]])
    else:
      lines = re.findall(r"([А-ЯЁ][а-яёА-ЯЁ\-, ]+): +(.*?)(?=\s*[А-ЯЁ][а-яёА-ЯЁ\-, ]+:|$)", episodes[i][j])
      thisquote = []
      for quoteparams in lines:
        if quoteparams[0] not in char_ids.values():
          char_ids[cur_chid] = quoteparams[0]
          char_ids_map[quoteparams[0]] = cur_chid
          cur_chid = cur_chid + 1
        thisquote.append([quoteparams[1], cur_id, char_ids_map[quoteparams[0]]])
      dialogues.append(thisquote)
    cur_id = cur_id + 1

Записываем токены

In [ ]:
with open('corpus_clean.txt', 'w') as file:
  for i in range(len(dialogues)):
    for j in range(len(dialogues[i])):
      file.write('\n'.join([str(elem) for elem in dialogues[i][j]]) + '\n')
    file.write('\n')

Записываем айди текстов

In [ ]:
texts_towrite = list(text_ids.items())
with open('text_ids.txt', 'w') as file:
  for i in range(len(texts_towrite)):
      file.write('\t'.join([str(elem) for elem in texts_towrite[i]]) + '\n')

Импорт спейси

In [ ]:
!pip install spacy -q
!python -m spacy download ru_core_news_sm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 41.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy

nlp = spacy.load("ru_core_news_sm")

Создаем словарь айди тегов

In [ ]:
spacypostags = ["ADJ", "ADP", "ADV", "AUX", "CONJ", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "PUNCT", "SCONJ", "SYM", "VERB", "X", "GER", "PRTCP"]
pos_ids = {}
pos_ids_map = {}
for i in range(len(spacypostags)):
  pos_ids[i] = spacypostags[i]
  pos_ids_map[spacypostags[i]] = i

Майстем для спорных случаев

In [ ]:
from pymystem3 import Mystem

In [ ]:
m = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


Список имен персонажей для тех случаев, где неправильно определяется лемма

In [ ]:
char_names = ['крош', 'ёжик', 'ежик', 'нюша', 'бараш', 'пин', 'кар-карыч', 'копатыч', 'лосяш', 'карыч', 'совунья']

Теггируем тоены. Знаки препинания кроме разделителей предложений (. ! ? и троеточие) присоединяем к предыдущему токену. Случаи типа "какой-то" записываем в один токен, "туманность-то" - в два. Иногда неправильно определялись леммы (например, у прилагательного в ж.р. или у имен персонажей в косвенных падежах) - это я стараюсь чинить по правилам, лемматизируя майстемом там, где не справился спейси

In [ ]:
def spacy_tagger(to_tag, text_id, char_id):
  global pos_ids_map, char_names
  toks_info = []
  doc_spacy = nlp(to_tag)
  tire = False
  for i in range(len(doc_spacy)):
    if doc_spacy[i].pos_ == 'PUNCT' and doc_spacy[i].text not in '!.?…':
      if len(toks_info) != 0:
        if len(toks_info[-1]) == 5:
          toks_info[-1].append(doc_spacy[i].text)
        else:
          toks_info[-1][-1] = toks_info[-1][-1] + doc_spacy[i].text
    elif doc_spacy[i].pos_ == 'PUNCT' and doc_spacy[i].text in '!.?…':
      if len(toks_info) != 0:
        if toks_info[-1][2] == 13:
          toks_info[-1][0] = toks_info[-1][0] + doc_spacy[i].text
          toks_info[-1][1] = toks_info[-1][1] + doc_spacy[i].text
        else:
          toks_info.append([doc_spacy[i].text, doc_spacy[i].text, 13, text_id, char_id])
      else:
        toks_info.append([doc_spacy[i].text, doc_spacy[i].text, 13, text_id, char_id])
    elif doc_spacy[i].pos_ != 'PUNCT' and doc_spacy[i].text == '-':
      tire = True
    elif doc_spacy[i].pos_ != 'SPACE':
      if tire:
        if pos_ids_map[doc_spacy[i].pos_] == toks_info[-1][2]:
          toks_info[-1][0] = toks_info[-1][0] + '-' + doc_spacy[i].text
          toks_info[-1][1] = toks_info[-1][1] + '-' + doc_spacy[i].lemma_
        else:
          toks_info.append([doc_spacy[i].text, doc_spacy[i].lemma_, pos_ids_map[doc_spacy[i].pos_], text_id, char_id])
        tire = False
      else:
        thepos = doc_spacy[i].pos_
        if thepos == 'VERB':
          if 'VerbForm=Conv' in doc_spacy[i].morph:
            thepos = 'GER'
          elif 'VerbForm=Part' in doc_spacy[i].morph:
            thepos = 'PRTCP'
        toks_info.append([doc_spacy[i].text, doc_spacy[i].lemma_, pos_ids_map[thepos], text_id, char_id])
        if 'Aspect=Imp' in doc_spacy[i].morph and toks_info[-1][0].lower() == toks_info[-1][1]:
          toks_info[-1][1] = m.lemmatize(doc_spacy[i].text)[0]
        elif toks_info[-1][2] == 12 and 'Case=Nom' not in doc_spacy[i].morph and toks_info[-1][0].lower() == toks_info[-1][1]:
          for name in char_names:
            if name in toks_info[-1][1] or name[:-1] in toks_info[-1][1]:
              toks_info[-1][1] = name
              break
        elif toks_info[-1][2] == 11 and 'Case=Nom' not in doc_spacy[i].morph and toks_info[-1][0].lower() == toks_info[-1][1]:
          toks_info[-1][1] = m.lemmatize(doc_spacy[i].text)[0]
        elif toks_info[-1][2] == 0 and toks_info[-1][1][-2:] == 'ая':
          toks_info[-1][1] = toks_info[-1][1][:-2] + 'ый'
        elif toks_info[-1][2] == 0 and toks_info[-1][1][-2:] == 'яя':
          toks_info[-1][1] = toks_info[-1][1][:-2] + 'ий'
  return toks_info

Теггируем

In [ ]:
tagged_quotes = []
spacy_counter = tqdm(total=len(dialogues), desc='Цитат размечено')
for i in range(len(dialogues)):
  for j in range(len(dialogues[i])):
    tagged_quotes.append(spacy_tagger(dialogues[i][j][0], dialogues[i][j][1], dialogues[i][j][2]))
  spacy_counter.update(1)

Цитат размечено:   0%|          | 0/1030 [00:00<?, ?it/s]

Записываем токены в формате 3грамм

In [ ]:
all_tagged_quotes = []
for i in range(len(tagged_quotes)):
  for j in range(len(tagged_quotes[i])):
    thetoken = []
    if j == 0:
      thetoken.append([None, None, None, None, None])
    elif tagged_quotes[i][j - 1][0] not in '!.?…':
      thetoken.append(tagged_quotes[i][j - 1])
    else:
      thetoken.append([None, None, None, None, None])
    thetoken.append(tagged_quotes[i][j])
    if j == len(tagged_quotes[i]) - 1:
      thetoken.append([None, None, None, None, None])
    else:
      thetoken.append(tagged_quotes[i][j + 1])
    if tagged_quotes[i][j][0] not in '!.?…':
      all_tagged_quotes.append(thetoken)

Всего 2624 токена

In [ ]:
len(all_tagged_quotes)

24624

Записываем их

In [ ]:
with open('corpus_tagged.txt', 'w') as file:
  for i in range(len(all_tagged_quotes)):
    for j in range(len(all_tagged_quotes[i])):
      file.write('\t'.join([str(elem) for elem in all_tagged_quotes[i][j]]) + '\n')
    file.write('\n')